<a href="https://colab.research.google.com/github/liamti5/UZH-Essentials-in-Text-and-Speech-Processing/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas
!pip install transformers
!pip install torch
!pip install sentencepiece
!pip install sacremoses

In [ ]:
import pandas as pd
import torch
import sentencepiece
import numpy as np


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
with open('/content/drive/My Drive/Colab/Uni/HS23/EITASP/reviews_en_translated.csv', 'r') as f:
  df_en_tr = pd.read_csv(f)

with open('/content/drive/My Drive/Colab/Uni/HS23/EITASP/reviews_de.csv', 'r') as f:
  df_de = pd.read_csv(f)

df_en_tr = df_en_tr.drop(columns=["translation"])
print(df_en_tr.head())

print(df_de.shape)

                        _id  \
0  5a5e48e1669270001a255a05   
1  5a5e4c87669270001a255a9e   
2  5a5e50f3be6796001a9d7316   
3  5a5e5127be6796001a9d7324   
4  5a5e5161be6796001a9d7337   

                                              review  \
0                              HSG too good for this   
1                             Winkelmann = Ehrenmann   
2  PROs: This module represents for the student t...   
3  Great Prof.! Best one so far. Easy to get good...   
4  The AC part is great but OC is trying to pack ...   

                                        translations  score university  \
0                                   HSG zu gut dafür      1        UZH   
1                             Winkelmann = Ehrenmann      5        UZH   
2                                                NaN      3        UZH   
3  Große Prof.! Beste bis jetzt. Einfach gute Not...      5        UZH   
4  Der AC-Teil ist toll, aber OC versucht, zu vie...      2        UZH   

     course  upvotes  downvotes

In [ ]:
df_en_tr = df_en_tr.drop(columns=["review"])
df_en_tr = df_en_tr.rename(columns={"translations": "review"})
print(df_en_tr.head())

                        _id  \
0  5a5e48e1669270001a255a05   
1  5a5e4c87669270001a255a9e   
2  5a5e50f3be6796001a9d7316   
3  5a5e5127be6796001a9d7324   
4  5a5e5161be6796001a9d7337   

                                              review  score university  \
0                                   HSG zu gut dafür      1        UZH   
1                             Winkelmann = Ehrenmann      5        UZH   
2                                                NaN      3        UZH   
3  Große Prof.! Beste bis jetzt. Einfach gute Not...      5        UZH   
4  Der AC-Teil ist toll, aber OC versucht, zu vie...      2        UZH   

     course  upvotes  downvotes           date  \
0  50044544      4.0       41.0  1548419537136   
1  50047387      1.0        0.0  1608566977197   
2  50772388      3.0        0.0  1578223557075   
3  50772226      0.0        1.0  1632840742122   
4  50740660      0.0        1.0  1626375246178   

                                          courseName  \
0          

In [ ]:
df_main = pd.concat([df_de, df_en_tr])
df_main.head()

,_id,review,score,university,course,upvotes,downvotes,date,courseName,courseNameShort
0,5a5b6dfbaf6c59001a536aeb,interessantes aber sehr aufwändiges Fach. Die ...,3,UZH,50314925,0.0,1.0,1562135719091,Asset Pricing (V+Ü),Asset Pricing
1,5a5b70abaf6c59001a536b07,Warum wird Moodle Verwendet & nicht OLAT? Anso...,4,UZH,50038004,4.0,8.0,1538035078776,Mathematik I (V+Ü) (Mathematics I),Mathematik I
2,5a5b729faf6c59001a536b13,"langwiilig, Management teil nutzlos, nur uswän...",2,UZH,50038000,0.0,1.0,1642090754318,Betriebswirtschaftslehre I (V + Ü) (Business A...,Betriebswirtschaftslehre I
3,5a5b7ae5af6c59001a536b80,In diesem Fach zeichnet man Flowcharts und Pro...,3,UZH,50330434,0.0,0.0,1515945190594,Wirtschaftsinformatik (V+Ü) (Business Informat...,Wirtschaftsinformatik
4,5a5b7d07af6c59001a536b87,weiss nöd warum so viel klaget. de dozent isch...,5,UZH,50030887,1.0,0.0,1655832259676,Statistik (V+Ü) (Statistics),Statistik


In [ ]:
df_main['review'].replace('', np.nan, inplace=True)
df_main.shape

(9725, 10)

In [ ]:
df_main.to_csv('/content/drive/My Drive/Colab/Uni/HS23/EITASP/main.csv', index=False)

In [ ]:
df_only_relevant = df_main[["review", "score", "university", "course", "upvotes", "downvotes", "date"]]
df_only_relevant.shape

(9725, 7)

In [ ]:
df_only_releveant = df_only_relevant.dropna(subset='review')
df_only_relevant.shape

(9725, 7)

In [ ]:
df_only_relevant['total_votes'] = df_only_relevant['upvotes'] - df_only_relevant['downvotes']
df_only_relevant['useful'] = (df_only_relevant['upvotes'] - df_only_relevant['downvotes'] > 0).astype(int)
print(df_only_relevant.head())

                                              review  score university  \
0  interessantes aber sehr aufwändiges Fach. Die ...      3        UZH   
1  Warum wird Moodle Verwendet & nicht OLAT? Anso...      4        UZH   
2  langwiilig, Management teil nutzlos, nur uswän...      2        UZH   
3  In diesem Fach zeichnet man Flowcharts und Pro...      3        UZH   
4  weiss nöd warum so viel klaget. de dozent isch...      5        UZH   

     course  upvotes  downvotes           date  total_votes  useful  
0  50314925      0.0        1.0  1562135719091         -1.0       0  
1  50038004      4.0        8.0  1538035078776         -4.0       0  
2  50038000      0.0        1.0  1642090754318         -1.0       0  
3  50330434      0.0        0.0  1515945190594          0.0       0  
4  50030887      1.0        0.0  1655832259676          1.0       1  


<ipython-input-12-fbecf17b553e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_only_relevant['total_votes'] = df_only_relevant['upvotes'] - df_only_relevant['downvotes']
<ipython-input-12-fbecf17b553e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_only_relevant['useful'] = (df_only_relevant['upvotes'] - df_only_relevant['downvotes'] > 0).astype(int)


In [ ]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("bert-base-german-cased")
model = AutoModel.from_pretrained("bert-base-german-cased")

In [ ]:
df_only_relevant['tokens'] = df_only_relevant['review'].astype(str).apply(tokenizer)

Token indices sequence length is longer than the specified maximum sequence length for this model (544 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
df_only_relevant
df_only_relevant.to_csv('/content/drive/My Drive/Colab/Uni/HS23/EITASP/tokenized.csv', index=False)

df = df_only_relevant

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

# Pad or truncate the tokenized sequences to a fixed length
max_length = 512  # You can adjust this based on the data
X_train_padded = np.array([x[:max_length] + [0]*(max_length - len(x)) for x in X_train])
X_test_padded = np.array([x[:max_length] + [0]*(max_length - len(x)) for x in X_test])

# Initialize and train the Logistic Regression model
log_reg = LogisticRegression(max_iter=1000, random_state=42)
log_reg.fit(X_train_padded, y_train)

# Make predictions on the test set
y_pred = log_reg.predict(X_test_padded)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

accuracy, classification_rep


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


(0.6370179948586119,
 '              precision    recall  f1-score   support\n\n           0       0.65      0.95      0.77      1264\n           1       0.38      0.06      0.10       681\n\n    accuracy                           0.64      1945\n   macro avg       0.52      0.50      0.44      1945\nweighted avg       0.56      0.64      0.54      1945\n')

In [ ]:
from transformers import BertForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
import torch
from sklearn.model_selection import train_test_split
import pandas as pd

# Extract 'input_ids' from the 'BatchEncoding' objects and labels
X = df['tokens'].apply(lambda x: x['input_ids'])
y = df['useful']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the model
model = BertForSequenceClassification.from_pretrained('google/bert_uncased_L-4_H-256_A-4', num_labels=2)

# Pad or truncate the tokenized sequences to a fixed length
max_length = 512  # You can adjust this based on the data
X_train_padded = [x[:max_length] + [0]*(max_length - len(x)) for x in X_train]
X_test_padded = [x[:max_length] + [0]*(max_length - len(x)) for x in X_test]

# Create DataLoader for training
train_dataset = TensorDataset(torch.tensor(X_train_padded, dtype=torch.long), torch.tensor(list(y_train), dtype=torch.long))
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

# Create DataLoader for testing
test_dataset = TensorDataset(torch.tensor(X_test_padded, dtype=torch.long), torch.tensor(list(y_test), dtype=torch.long))
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)


# Define optimizer and loss function
optimizer = optim.AdamW(model.parameters(), lr=1e-5)
loss_fn = torch.nn.CrossEntropyLoss()

# Training loop
for epoch in range(1):  # Number of epochs
    model.train()
    for i, batch in enumerate(train_loader):
        input_ids, labels = batch
        optimizer.zero_grad()
        outputs = model(input_ids)[0]
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()

        if i % 100 == 0:
            print(f"Epoch: {epoch}, Iteration: {i}, Loss: {loss.item()}")

# Evaluation
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for input_ids, labels in test_loader:
        outputs = model(input_ids)[0]
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total}")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-4_H-256_A-4 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch: 0, Iteration: 0, Loss: 0.7204047441482544
Epoch: 0, Iteration: 100, Loss: 0.603890061378479
Epoch: 0, Iteration: 200, Loss: 0.7337659001350403
Epoch: 0, Iteration: 300, Loss: 0.8386585116386414
Epoch: 0, Iteration: 400, Loss: 0.5431475043296814
Epoch: 0, Iteration: 500, Loss: 0.9555847644805908
Epoch: 0, Iteration: 600, Loss: 0.5982544422149658
Epoch: 0, Iteration: 700, Loss: 0.7859146595001221
Epoch: 0, Iteration: 800, Loss: 0.6921529769897461
Epoch: 0, Iteration: 900, Loss: 0.5781229138374329
Epoch: 0, Iteration: 1000, Loss: 0.9712679386138916
Epoch: 0, Iteration: 1100, Loss: 0.39477503299713135
Epoch: 0, Iteration: 1200, Loss: 0.4791734218597412
Epoch: 0, Iteration: 1300, Loss: 0.5578685402870178
Epoch: 0, Iteration: 1400, Loss: 0.7327491044998169
Epoch: 0, Iteration: 1500, Loss: 0.43951284885406494
Epoch: 0, Iteration: 1600, Loss: 0.5563516616821289
Epoch: 0, Iteration: 1700, Loss: 0.7357560992240906
Epoch: 0, Iteration: 1800, Loss: 0.8139868974685669
Epoch: 0, Iteration: 19